`Prerequisites`

To complete this exercise, you must have access to the `little_lemon` database. As an authorized user, you need to establish a connection between Python and the database via the connector API and create a cursor object

Once the connection is established, and you have a cursor object, you can select the database little_lemon and print the names of the tables

You should have the four created tables from previous labs in your database envionrment


#### **Learning objectives:**
* Join and retrieve the required data from different tables in a MySQL database using `JOIN operations in Python`.



`Scenario:`

Little Lemon needs to carry out the following tasks with their datasets:
* Determine the final bill amount for each customer who attended the restaurant today 
* Identify the type of cuisine that each item in their menu belongs to.

In [1]:
import mysql.connector as connector

try:
    connection=connector.connect(user="root", password="")
    print('Connection established!')
except connector.Error as e:
    print(f"""
    Error code : {e.errno}|
    Error message : {e.msg}
    """)

Connection established!


In [2]:
cursor = connection.cursor()

# select little_lemon db
cursor.execute('USE little_lemon;')

# print out db in use
f'We are connected to : {connection.database}'

'We are connected to : little_lemon'

#### A Few More Prereqs
* Function to look at query returns
* Addition to Menus following previous deletion

In [8]:
def query_return(crs, query):
    # execute query
    crs.execute(query)
    print(crs.column_names)
    for row in crs.fetchall():
        print(row)

In [10]:
# insert into Menu
greek_insert = """
INSERT INTO Menus
VALUES 
(1, 1, 'Greek'),
(1,7, 'Greek'),
(1,10, 'Greek'),
(1,13, 'Greek')
"""
# execute
cursor.execute(greek_insert)
# commit to db
connection.commit()

### **Task 1:**

Little Lemon needs the following information for each of the items on their menu:
* The name of each item in the menu,
* Each menu item’s type,
* Each menu item’s cuisine,
* and the price of each item in the menu.

In [11]:
menu_itms_qry = """
SELECT
    mi.Name, mi.Type, mn.Cuisine, mi.Price
FROM MenuItems AS mi
INNER JOIN Menus AS mn
    ON mi.ItemID = mn.ItemID
"""
query_return(cursor, menu_itms_qry)

('Name', 'Type', 'Cuisine', 'Price')
('Olives', 'Starters', 'Greek', 5)
('Greek salad', 'Main Courses', 'Greek', 15)
('Greek yoghurt', 'Desserts', 'Greek', 7)
('Athens White wine', 'Drinks', 'Greek', 25)
('Minestrone', 'Starters', 'Italian', 8)
('Pizza', 'Main Courses', 'Italian', 15)
('Cheesecake', 'Desserts', 'Italian', 4)
('Turkish Coffee', 'Drinks', 'Italian', 10)
('Falafel', 'Starters', 'Turkish', 7)
('Ice cream', 'Desserts', 'Turkish', 6)
('Turkish Coffee', 'Drinks', 'Turkish', 10)
('Kabasa', 'Main Courses', 'Turkish', 17)


### **Task 2:**

Little Lemon notice that there are several menu items in the MenuItems records that are absent from the Menus table.

* Help Little Lemon to identify the items that are present in the MenuItems records but absent from the Menus table 
* Columns : [Item_Name, Item_Type, Price, Cuisine]

In [15]:
all_mi_on_menus = """
SELECT 
    mi.Name, mi.Type, mi.Price, 
    COALESCE(mn.Cuisine, 'No Cuisine Value for MenuItem') AS Cuisine_Null
FROM MenuItems AS mi
LEFT JOIN Menus AS mn
USING(ItemID)
ORDER BY mi.ItemID;
"""
query_return(cursor, all_mi_on_menus)

('Name', 'Type', 'Price', 'Cuisine_Null')
('Olives', 'Starters', 5, 'Greek')
('Flatbread', 'Starters', 5, 'No Cuisine Value for MenuItem')
('Minestrone', 'Starters', 8, 'Italian')
('Tomato bread', 'Starters', 8, 'No Cuisine Value for MenuItem')
('Falafel', 'Starters', 7, 'Turkish')
('Hummus', 'Starters', 5, 'No Cuisine Value for MenuItem')
('Greek salad', 'Main Courses', 15, 'Greek')
('Bean soup', 'Main Courses', 12, 'No Cuisine Value for MenuItem')
('Pizza', 'Main Courses', 15, 'Italian')
('Greek yoghurt', 'Desserts', 7, 'Greek')
('Ice cream', 'Desserts', 6, 'Turkish')
('Cheesecake', 'Desserts', 4, 'Italian')
('Athens White wine', 'Drinks', 25, 'Greek')
('Corfu Red Wine', 'Drinks', 30, 'No Cuisine Value for MenuItem')
('Turkish Coffee', 'Drinks', 10, 'Italian')
('Turkish Coffee', 'Drinks', 10, 'Turkish')
('Kabasa', 'Main Courses', 17, 'Turkish')


In [24]:
# Let's look just at the MenuItems w/o a Value in Menus
mi_not_on_menus = """
SELECT 
    mi.Name, mi.Type, mi.Price, 
    COALESCE(mn.Cuisine, 'No Cuisine Value for MenuItem') AS Cuisine_Null
FROM MenuItems AS mi
LEFT JOIN Menus AS mn
USING(ItemID)
WHERE mn.Cuisine IS NULL
ORDER BY mi.ItemID;
"""
query_return(cursor, mi_not_on_menus)

('Name', 'Type', 'Price', 'Cuisine_Null')
('Flatbread', 'Starters', 5, 'No Cuisine Value for MenuItem')
('Tomato bread', 'Starters', 8, 'No Cuisine Value for MenuItem')
('Hummus', 'Starters', 5, 'No Cuisine Value for MenuItem')
('Bean soup', 'Main Courses', 12, 'No Cuisine Value for MenuItem')
('Corfu Red Wine', 'Drinks', 30, 'No Cuisine Value for MenuItem')


### **Task 3**
Little Lemon restaurant need you to help them retrieve the following information from the Bookings and the Orders tables in their MySQL database using Python:
* Booking ID,
* Table number,
* Guest first name,
* And Bill amount.

In [25]:
## want all booking regardless of orders
booking_orders = """
SELECT
    b.BookingID,
    b.TableNo,
    b.GuestFirstName,
    o.BillAmount,
    b.EmployeeID AS Server
FROM Bookings AS b
LEFT JOIN Orders AS o
USING(BookingID) # share same booking iD
ORDER BY o.BillAmount DESC
"""
query_return(cursor, booking_orders)

('BookingID', 'TableNo', 'GuestFirstName', 'BillAmount', 'Server')
(1, 12, 'Anna', 86, 1)
(5, 5, 'Hiroki', 43, 2)
(4, 15, 'Marcos', 40, 4)
(2, 11, 'Joakim', 37, 6)
(3, 19, 'Vanessa', 37, 3)
(6, 10, 'Diana', None, 5)


* Just a little note, the passed string does notice the comment and still runs too